 <pre>
 ____   ____    _    ____  _____                          _      _     
|  _ \ / ___|  / \  / ___|| ____|     _ __ ___   ___   __| | ___| |___ 
| | | | |     / _ \ \___ \|  _| _____| '_ ` _ \ / _ \ / _` |/ _ \ / __|
| |_| | |___ / ___ \ ___) | |__|_____| | | | | | (_) | (_| |  __/ \__ \
|____/ \____/_/   \_\____/|_____|    |_| |_| |_|\___/ \__,_|\___|_|___/
                                                                        
</pre>

# DCASE-models Notebooks
Python Notebooks for [DCASE-models](https://github.com/pzinemanas/DCASE-models)

---

### About 
This Notebook reproduces the results for **Sound Event Detection (SED)** presented in:
<ul>
<li><a href="http://www.justinsalamon.com/uploads/4/3/9/4/4394963/salamon_scaper_waspaa_2017.pdf"><strong>
    Scaper: A Library for Soundscape Synthesis and Augmentation</strong></a>
    J. Salamon, D. MacConnell, M. Cartwright, P. Li, and J. P. Bello.
    In IEEE Workshop on Applications of Signal Processing to
    Audio and Acoustics (WASPAA), New Paltz, NY, USA, Oct. 2017.
    <br>
   <a type="button" class="btn btn-default btn-xs" target="_blank" href="http://www.justinsalamon.com/uploads/4/3/9/4/4394963/salamon_scaper_waspaa_2017.pdf"> PDF </a>
   <a type="button" class="btn btn-default btn-xs" target="_blank" href="https://ieeexplore.ieee.org/document/8170052"> IEEE</a>
    </li>   
</ul>

### Overview

The paper introduces [Scaper](https://github.com/justinsalamon/scaper), an open-source library for soundscape synthesis and augmentation. To illustrate the potential of the library, the authors generate a dataset of 10,000 sound-scapes, namely [URBAN-SED](http://urbansed.weebly.com/), and use it to compare the performance of two state-of-the-art algorithms for sound event detection:
- the Convolutional Recurrent Neural Net-work (CRNN) proposed by Cakir et al. [[C-CRNN]](https://ieeexplore.ieee.org/document/7933050)
- an adaptation of the Convolutional Neural Network (CNN) proposed by Salamon and Bello [[SB-CNN]](http://ieeexplore.ieee.org/document/7829341/)

### Organization

The Notebook is organized into the following sections.
* [1. Load parameters](#LoadParameters)
* [2. Init data generator and extract features](#ExtractFeatures)
* [3. Load data](#LoadData)
* [4. Initialize model](#InitModel)
* [5. Train model](#TrainModel)
* [6. Evaluate model](#EvaluateModel)

In [36]:
%load_ext autoreload
%autoreload 2
rootdir_path = '../../'
import sys
import os
import json
import warnings
import glob
import numpy as np
import argparse

sys.path.append(rootdir_path)
from dcase_models.utils.files import load_json, mkdir_if_not_exists
from dcase_models.data.data_generator import DataGenerator
from dcase_models.data.datasets import URBAN_SED
from dcase_models.data.features import MelSpectrogram
from dcase_models.model.models import SB_CNN_SED
from dcase_models.data.scaler import Scaler
from dcase_models.utils.files import load_json

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a id="LoadeParameters"></a>
## 1. Load parameters

Dataset, feature extraction and training parameters are stored in a json file on the root directory.

In [37]:
# load all parameters from json file
params = load_json(os.path.join(rootdir_path, 'parameters.json'))
# set the dataset we are going to use
dataset = 'URBAN_SED'

# get dataset parameters
params_dataset = params["datasets"][dataset]

# get feature extraction parameters
params_features = params["features"]

# get training parameters
params_train = params["train"]

Check that the values of the parameters are correct.

In [38]:
# print the dataset parameters 
print("Dataset Parameters:\n", json.dumps(params_dataset, indent=4, sort_keys=True))
# print feature extraction parameters 
print("Features' Parameters:\n",json.dumps(params_features, indent=4, sort_keys=True))
# print training parameters 
print("Training Parameters:\n",json.dumps(params_train, indent=4, sort_keys=True))



Dataset Parameters:
 {
    "dataset_path": "datasets/URBAN-SED_v2.0.0",
    "evaluation_mode": "train-validate-test"
}
Features' Parameters:
 {
    "MelSpectrogram": {
        "mel_bands": 64,
        "n_fft": 1024
    },
    "Openl3": {
        "content_type": "env",
        "embedding_size": 512,
        "input_repr": "mel256"
    },
    "Spectrogram": {
        "n_fft": 1024
    },
    "audio_hop": 690,
    "audio_win": 1024,
    "sequence_hop_time": 1.0,
    "sequence_time": 2.0,
    "sr": 22050
}
Training Parameters:
 {
    "batch_size": 256,
    "considered_improvement": 0,
    "early_stopping": 30,
    "epochs": 50,
    "learning_rate": 0.001,
    "optimizer": "Adam",
    "verbose": 1
}


<a id="ExtractFeatures"></a>
## 2. Initialize Data Generator and Extract features

Initialize Feature Extractor and Data Generator.

In [39]:
# Initialize Feature Extractor
feature_extractor = MelSpectrogram(sequence_time=params_features['sequence_time'], \
                                            sequence_hop_time=params_features['sequence_hop_time'], 
                                            audio_win=params_features['audio_win'], 
                                            audio_hop=params_features['audio_hop'], 
                                            n_fft=params_features['Spectrogram']['n_fft'], 
                                            sr=params_features['sr'])

print(feature_extractor.get_shape())

(10, 64, 128)


In [41]:
# Initialize Data Generator as an instance of URBAN_SED
dataset = URBAN_SED(os.path.join(rootdir_path, params_dataset["dataset_path"]))

Check if dataset exists, and download it if doesn't.

In [ ]:
dataset.download()

Initialize data generator

In [42]:
data_generator = DataGenerator(dataset, feature_extractor)

AttributeError: dataset has to be an instance of Dataset or similar

Extract the features (if they were not extracted before).

In [ ]:
if not data_generator.check_if_features_extracted():
    data_generator.extract_features()
print('Done!')

<a id="LoadData"></a>
## 3. Load data

In [ ]:
print('Loading data... ')
data_generator.load_data()
print('Done!')

And also fit a scaler and transform the training data.

In [ ]:
fold_test = 'test'

X_train, Y_train, X_val, Y_val = data_generator.get_data_for_training(fold_test, 
                                                                      evaluation_mode='train-validate-test')
scaler = Scaler(normalizer=normalizer)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [ ]:
print(X_train.shape, Y_train.shape, X_val[0].shape, Y_val[0].shape)

<a id="InitModel"></a>
## 4. Initialize model

In [ ]:
n_frames_cnn = X_train.shape[1]
n_freq_cnn = X_train.shape[2]
n_classes = Y_train.shape[1]
print(n_frames_cnn, n_freq_cnn, n_classes)
metrics = ['F1', 'ER']

model_container = SB_CNN_SED(model=None, model_path=None, n_classes=n_classes, 
                             n_frames_cnn=n_frames_cnn, n_freq_cnn=n_freq_cnn,
                             metrics=metrics)

model_container.model.summary()

<a id="TrainModel"></a>
## 5. Train model

In [ ]:
exp_folder = './'

model_container.train(X_train, Y_train, X_val, Y_val, weights_path=exp_folder, **params_train)

<a id="EvaluateModel"></a>
## 6. Evaluate Model

In [ ]:
# Load best_weights
model_container.load_model_weights(exp_folder)

# Test model
X_test, Y_test = data_generator.get_data_for_testing(fold_test)
X_test = scaler.transform(X_test)
result  s = model_container.evaluate(X_test, Y_test)

print(results['F1'], results['ER'])